### **Using SNAP with ArgoWorkflows**

This notebook will show how a simple example of how to use ESA`s SNAP software in a workflow.

**First some imports and global settings**

In [ ]:
from dotenv import load_dotenv
import os
from hera.workflows import  models, script, Artifact, DAG, NoneArchiveStrategy, Workflow, Container
from hera.shared import global_config

global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "<YOUR NAMESPACE>"
global_config.token = "<YOUR TOKEN>"
global_config.image = "ghcr.io/eodcgmbh/cluster_image:2025.2.0"

**Setting up Volume**

We need to set up our the connection to the volume we want to write to. 

In [ ]:
nfs_volume = models.Volume(
    name="eodc-mount",
    persistent_volume_claim={"claimName": "eodc-nfs-claim"},
    )

security_context = {"runAsUser": <YOUR UID>,
                    "runAsGroup": <YOUR GID>}

**Writing scripts**

For this example, we will create a simple Land-Water mask for a Sentinel-1 SIG0 image and write the resulting image to our path in the EODC NFS.

First, we download the image from the EODC STAC collection, and save the image as an Artifact to pass down to the next steps. We choose a fixed time and spatial extent, but you could also parameterize this.

In [ ]:
@script(outputs=Artifact(name="tif-image", path="/tmp/input.tif", archive=NoneArchiveStrategy()))
def download_stac_item():
    import pystac_client as pc
    from urllib.request import urlretrieve

    pc_client = pc.Client.open("https://stac.eodc.eu/api/v1")

    time_range = "2024-01-01/2024-01-30"

    search = pc_client.search(
        collections=["SENTINEL1_SIG0_20M"],
        datetime=time_range,
        query={"Equi7_TileID": {"eq": "EU020M_E051N018T3"}},
        max_items=1
    )

    items = search.item_collection()
    
    url = items[0].assets["VH"].href

    urlretrieve(url, "/tmp/input.tif")

Next, we need to get the xml to tell SNAP how to do the processing. You can get the xml from the SNAP software by creating a processing graph and exporting the graph as an xml. When copying the xml you need to make sure to adjust the input and output path accordingly. The xml will also be saved as an Artifact to be passed down to the next step.

In [ ]:
@script(outputs=Artifact(name="processing-graph", path="/tmp/processing_graph.xml", archive=NoneArchiveStrategy()))
def snap_graph():
    output_path = "/eodc/private/tempearth/masked/LW_masked.tif"
    input_artifact = "/tmp/input.tif"

    xml = f"""
                <graph id="Graph">
                <version>1.0</version>
                <node id="Read">
                <operator>Read</operator>
                <sources/>
                <parameters class="com.bc.ceres.binding.dom.XppDomElement">
                <useAdvancedOptions>true</useAdvancedOptions>
                <file>{input_artifact}</file>
                <copyMetadata>true</copyMetadata>
                <bandNames/>
                <pixelRegion>0,0,15000,15000</pixelRegion>
                <maskNames/>
                </parameters>
                </node>
                <node id="LandWaterMask">
                <operator>LandWaterMask</operator>
                <sources>
                <sourceProduct refid="Read"/>
                </sources>
                <parameters class="com.bc.ceres.binding.dom.XppDomElement">
                <resolution>50</resolution>
                <subSamplingFactorX>1</subSamplingFactorX>
                <subSamplingFactorY>1</subSamplingFactorY>
                </parameters>
                </node>
                <node id="Write">
                <operator>Write</operator>
                <sources>
                <sourceProduct refid="LandWaterMask"/>
                </sources>
                <parameters class="com.bc.ceres.binding.dom.XppDomElement">
                <file>{output_path}</file>
                <formatName>GeoTIFF</formatName>
                </parameters>
                </node>
                <applicationData id="Presentation">
                <Description/>
                <node id="Read">
                <displayPosition x="89.0" y="174.0"/>
                </node>
                <node id="LandWaterMask">
                <displayPosition x="224.0" y="191.0"/>
                </node>
                <node id="Write">
                <displayPosition x="455.0" y="135.0"/>
                </node>
                </applicationData>
                </graph>
                """
    
    with open("/tmp/processing_graph.xml", "w") as f:
        f.write(xml)

Finally, we can do the SNAP processing. The [image](https://github.com/oscipal/snap_image) given here has SNAP v12 installed, you can also create your own image, just beware that SNAP needs to be able to write an auxillary data when running for the first time. In this image the path to the auxData is set to go to the EODC NFS, if you don't have read access to the folder in line 19 of the dockerfile, you need to adjust your image accordingly.

In [ ]:
snap_processing = Container(
    name="snap-processing",
    image="ghcr.io/oscipal/snap_image",
    command=["gpt"],
    args=["/tmp/processing_graph.xml"],
    inputs=[Artifact(name="tif-image", path="/tmp/input.tif"),
            Artifact(name="processing-graph", path="/tmp/processing_graph.xml")],
    volume_mounts=[models.VolumeMount(name="eodc-mount", mount_path="/eodc")],
    )

**Creating the Workflow**

In [ ]:
with Workflow(
    generate_name="snap-processing-",
    volumes = [nfs_volume],
    security_context=security_context,
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        step1 = download_stac_item()
        step2 = snap_graph()
        step3 = snap_processing(arguments=[step1.get_artifact("tif-image").with_name("tif-image"),
                                           step2.get_artifact("processing-graph").with_name("processing-graph")])
        
        step1 >> step2 >> step3

**Submitting the Workflow**

In [ ]:
w.create()